In [91]:
import os

In [92]:
import warnings
warnings.filterwarnings("ignore")

In [93]:
from dotenv import load_dotenv
load_dotenv()

True

In [94]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_TRACING_V2"] = LANGCHAIN_TRACING_V2

LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

# DATA INGESTION

LOADING OF DATA 

In [39]:
import requests
from langchain.document_loaders import TextLoader , WebBaseLoader
from langchain.vectorstores import FAISS

In [23]:
url = "https://en.wikipedia.org/wiki/State_of_the_Union"
loader = WebBaseLoader(url)
docs = loader.load()
response = docs[0].page_content

In [26]:
print(type(response))

<class 'str'>


In [95]:
with open("/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt" , "w") as f:
    f.write(response)

In [97]:
text_loader = TextLoader('/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt')
document = text_loader.load()

print(document)

[Document(metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='State of the Union (Four Freedoms)(January 6, 1941)\n\nFranklin Delano Roosevelt\'s January 6,1941 State of the Union Address,\n  introducing the theme of theFour Freedoms (starting at 32:02)\nProblems playing this file? See media help.\n\nPrior to 1934, the annual message was delivered at the end of the calendar year, in December. The ratification of the 20th Amendment on January 23, 1933, changed the opening of Congress from early March to early January, affecting the delivery of the annual message. Since 1934, the message or address has been delivered to Congress early in the calendar year.[12]\nThe Twentieth Amendment also established January 20 as the beginning of the presidential term. In years when a new president is inaugurated, the outgoing president may deliver a final State of the Union message, but none has done so since Jimmy Carter sent a written message in 1981. In 

CHUNKING OF DATA

In [98]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#chunkviz.up.railway.app

In [99]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100 , chunk_overlap = 50 )

In [100]:
text_chunks = text_splitter.split_documents(documents=document)
print(text_chunks)
print(len(text_chunks))
print(text_chunks[3].page_content)

[Document(metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='State of the Union (Four Freedoms)(January 6, 1941)'), Document(metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content="Franklin Delano Roosevelt's January 6,1941 State of the Union Address,"), Document(metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='introducing the theme of theFour Freedoms (starting at 32:02)'), Document(metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='Problems playing this file? See media help.'), Document(metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='Prior to 1934, the annual message was delivered at the end of the calendar year, in December. The'), Document(metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='at the end of the cal

Creating Embeddings

In [101]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x10fcce5e0>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [104]:
import sys
!{sys.executable} -m pip install faiss-cpu


  Using cached faiss_cpu-1.10.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (4.4 kB)
Using cached faiss_cpu-1.10.0-cp39-cp39-macosx_11_0_arm64.whl (3.2 MB)


In [105]:
import faiss
print(faiss.__version__)


1.10.0


In [106]:
from langchain.vectorstores import FAISS
vector_store = FAISS.from_documents( text_chunks , embedding=embeddings)

Data Retrieval

In [107]:
retriever = vector_store.as_retriever()

In [108]:
from langchain.prompts import ChatPromptTemplate , PromptTemplate

In [109]:
chat_template = ChatPromptTemplate([
     ('system',"You are an assisatnt for question-answering tasks."),
     ('human' , """ Use the following pieces of retireved context to answer the question. 
                    If you don't knwo the answer , just say you don't know.
                    Use ten sentence maximum and keep teh content concise.
                    Question : {question}
                    Context  : {context}
                    Answer : """)
])

In [110]:
template = """ 

    You are an assisatnt for question-answering tasks.
    
    Use the following pieces of retireved context to answer the question.
    If you don't knwo the answer , just say you don't know.
    Use ten sentence maximum and keep teh content concise.

    Question : {question}
    Context  : {context}

    Answer : 

 """

prompt = ChatPromptTemplate.from_template(template)

DATA GENERATION

In [112]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-1.5-pro" , temperature = 0 , max_tokens=1000)


In [115]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

parser = StrOutputParser()

rag_chain = {"context" : retriever , "question" : RunnablePassthrough()} | prompt | llm | parser

# Internally it is like
# {
#     "context": retriever.invoke("What is the capital of France?"),
#     "question": "What is the capital of France?"
# }


In [124]:
input1 = {
    "context": retriever.invoke("What is the capital of France?"),
    "question": "What is the capital of France?"
}

print(input1)

input2 = prompt.invoke(input1)

print(input2)

input3 = llm.invoke(input2)

print(input3)

output = parser.invoke(input3)

print(output)

{'context': [Document(id='602173ca-d887-4f88-bb38-62a324ba522d', metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='^ Briançon, Pierre (July 3, 2017). "Macron lacks fire in first \'state of the union\' à la'), Document(id='3eb20870-69bf-4e5d-a31d-32c734a75a86', metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='The State of the Union model has also been adopted by the European Union.[54] In France, President'), Document(id='18f9ec5c-68a1-48fa-88db-ae79e0843839', metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='lacks fire in first \'state of the union\' à la française". Politico. Paris. Archived from the'), Document(id='38efed7f-20a7-4b36-a043-a36a0f8dea57', metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='la française". Politico. Paris. Archived from the original on September 9, 2024. Retrieved')], 'q

In [120]:

context_temp = retriever.invoke("How is US supporting ukraine economically and militarily")
print(context_temp)
print(len(context_temp))
print(context_temp[0].page_content)

[Document(id='0a9dc2ad-6330-4de2-92da-4065c8d0d5ba', metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='became a defining moment in the foreign policy of the United States and one of its longest-standing'), Document(id='34b20b18-7055-47a5-ae50-e2ce0e53510c', metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='^ a b c d Blitzer, Wolf; Malveaux, Suzanne; Acosta, Jim; Yellin, Jessica; King, John; Borger,'), Document(id='4f3c7ef9-fe1e-451b-80d0-5cf09b6b6744', metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content='^ a b c Desjardins, Lisa (January 30, 2018). "The word nearly every president uses to describe the'), Document(id='1bced14b-8f63-4cd6-a1b3-e19811c7254c', metadata={'source': '/Users/akashjain/Desktop/LangGraph/Util/state_of_union.txt'}, page_content="^ a b c d e J. Shagan, Colleen (January 16, 2014). The President's State of the Union Address:")]
4
b

In [122]:
response = rag_chain.invoke("How is US supporting ukraine economically and militarily")
print(response)

This document set does not contain information about US support for Ukraine. Therefore, I cannot answer your question.


In [123]:
response = rag_chain.invoke("what is the protocol of entering the chamber")
print(response)

The Sergeant at Arms announces the President's entrance into the House chamber.  This occurs when the President is ready.  The Sergeant at Arms stands just inside the chamber doors.  Additional details about the protocol are not provided in the given text.
